In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import pickle

In [162]:
page_link = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
page_response = requests.get(page_link, timeout=1000)
page_content = BeautifulSoup(page_response.content)

div_list = page_content.find(id='constituents').find_all('tr')

In [186]:
# Get S&P 500 Constituents
co_name, co_ticker, co_link, co_sector, co_sub_sector, co_address = [], [], [], [], [], []

for idx, row in enumerate(div_list[1:]):
    co_name.append(row.find_all('td')[1].text)
    co_ticker.append(row.find_all('td')[0].text.replace('\n',''))
    co_link.append('https://en.wikipedia.org/' + row.find_all('td')[1].find('a').get('href'))
    co_sector.append(row.find_all('td')[3].text)
    co_sub_sector.append(row.find_all('td')[4].text)
    co_address.append(row.find_all('td')[5].text)
    
extracted_df = pd.DataFrame([co_name, co_ticker, co_link, co_sector, co_sub_sector, co_address]).T
extracted_df.rename({0: 'Name',
                     1: 'Ticker',
                     2: 'Wiki Link',
                     3: 'Sector',
                     4: 'Sub Sector',
                     5: 'HQ Location'}, 
                    axis=1, inplace=True)

In [213]:
# Scrape each company's wikipedia page for text

body_text_list = []

for i, row in extracted_df.iterrows():
    page_link = row['Wiki Link']
    page_response = requests.get(page_link, timeout=1000)
    page_content = BeautifulSoup(page_response.content)

    body_text = ''
    for i in page_content.find_all('p'):
        body_text += i.text
    body_text_list.append(body_text)

In [214]:
extracted_df['Body Text'] = body_text_list

In [216]:
extracted_df.head()

,Name,Ticker,Wiki Link,Sector,Sub Sector,HQ Location,Body Text
0,3M,MMM,https://en.wikipedia.org//wiki/3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",\nThe 3M Company is an American multinational ...
1,Abbott Laboratories,ABT,https://en.wikipedia.org//wiki/Abbott_Laborato...,Health Care,Health Care Equipment,"North Chicago, Illinois",\nAbbott Laboratories is an American multinati...
2,AbbVie,ABBV,https://en.wikipedia.org//wiki/AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",AbbVie is an American publicly traded biopharm...
3,Abiomed,ABMD,https://en.wikipedia.org//wiki/Abiomed,Health Care,Health Care Equipment,"Danvers, Massachusetts",Abiomed is a publicly-traded medical devices c...
4,Accenture,ACN,https://en.wikipedia.org//wiki/Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",\nAccenture plc is an Irish-based multinationa...
